# 04. Sample Prediction

In [1]:
import pickle
import pandas as pd
import re
import string
import unicodedata
import nltk
from bs4 import BeautifulSoup
from emo_unicode import UNICODE_EMO, EMOTICONS
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

In [2]:
# tf-idf object from 03_feature_engineering
tfidf_path = '../03_feature_engineering/03_pickle/03_tfidf.pickle'
with open(tfidf_path, 'rb') as tfidf_path:
    tfidf = pickle.load(tfidf_path)

# trained model (rocchio) from 04_model_training
model_path = '../04_model_training/04_pickle/04_model_rocchio.pickle'
with open(model_path, 'rb') as model_path:
    roc = pickle.load(model_path)

# trained model (knn) from 04_model_training
model_path = '../04_model_training/04_pickle/04_model_knn.pickle'
with open(model_path, 'rb') as model_path:
    knn = pickle.load(model_path)

In [3]:
# OPD mapping dictionary
category_codes = {
    'DINAS KEPENDUDUKAN DAN PENCATATAN SIPIL KOTA MALANG': 0,
    'DINAS PEKERJAAN UMUM DAN PENATAAN RUANG KOTA MALANG': 1,
    'DINAS LINGKUNGAN HIDUP KOTA MALANG': 2,
    'DINAS PENDIDIKAN KOTA MALANG': 3,
    'DINAS PERUMAHAN DAN KAWASAN PERMUKIMAN KOTA MALANG': 4,
    'SATUAN POLISI PAMONG PRAJA KOTA MALANG': 5,
    'DINAS PERHUBUNGAN KOTA MALANG': 6
}

In [4]:
def text_preprocessing(text):
    text = text.strip().lower()
    text = re.sub(r'https?://\S+|www\.\S+', '', text) # remove url
    text = re.sub('\S*@\S*\s?', '', text) # remove email
    text = re.sub('\[[^]]*\]', '', text) # remove beetwen square brackets []
    text = re.sub('[-+]?[0-9]+', '', text) # remove number
    emoticon_pattern = re.compile(u'(' + u'|'.join(k for k in EMOTICONS) + u')')
    emoji_pattern = re.compile("["
                               u"\U0001F600-\U0001F64F" # emoticons
                               u"\U0001F300-\U0001F5FF" # symbols & pictographs
                               u"\U0001F680-\U0001F6FF" # transport & map symbols
                               u"\U0001F1E0-\U0001F1FF" # flags (iOS)
                               u"\U00002500-\U00002BEF" # chinese char
                               "]+", flags=re.UNICODE)
    text = emoji_pattern.sub(r'', text) # remove emoji
    text = emoticon_pattern.sub(r'', text) # remove emoticon
    text = re.sub(r'[^\w\s]', '', text) # remove punctuation
    text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8', 'ignore') # remove non-ascii
    normal_word_path = pd.read_csv('../00_data/key_norm.csv')
    text = ' '.join([normal_word_path[normal_word_path['singkat'] == word]['hasil'].values[0] 
    if (normal_word_path['singkat'] == word).any() else word for word in text.split()])

    factory = StemmerFactory()
    stemmer = factory.create_stemmer()
    text = stemmer.stem(text)

    words = nltk.word_tokenize(text) # tokonize

    stopword = stopwords.words('indonesian')
    more_stopword = ['daring', 'online', 'pd'] # add more stopword to default corpus
    stop_factory = stopword + more_stopword
        
    clean_words = []
    for word in words:
        if word not in stop_factory:
            clean_words.append(word)
    words = clean_words
    words = ' '.join(words) # join
    return words
    
def create_features(text):
    df = pd.DataFrame(columns=['ticket'])
    df.loc[0] = text
    df['ticket'] = df['ticket'].apply(text_preprocessing)

    features = tfidf.transform(df['ticket']).toarray()
    return features

def get_category_name(category_id):
    for category, id_ in category_codes.items():    
        if id_ == category_id:
            return category

def predict_from_text(text):
    # Predict using the input model
    roc_prediction = roc.predict(create_features(text))[0]
    knn_prediction = knn.predict(create_features(text))[0]
    
    # Return result
    roc_category = get_category_name(roc_prediction)
    knn_category = get_category_name(knn_prediction)
    
    print("The predicted category using the rocchio model is %s." %(roc_category))
    print("The predicted category using the knn model is %s." %(knn_category))
    # print("The conditional probability is: %a" %(prediction_proba.max()*100))

In [9]:
text = 'Selamat pagi, saya dari orang tua dari siswa yang sekolah di SMAN 1 Bululawang. 1. Saya ingin bertanya apakah memang SMAN di KAB MALANG itu memiliki peraturan yang menyatakn di setiap kenaikan kelas ( Tiap Tahun ) HARUS daftar ulang dan membayar biaya daftar ulang sebesar Rp 700.000,- / tahunnya / kenaikan kelas ?! karena di sekolah anak saya harus melakukan dan membayar biaya daftar ulang tesebut ketika kenaikan kelas. Setau saya SMAN di KOTA MALANG tidak pernah ada yang Namanya daftar ulang di setiap kenaikan kelas kalua naik kelas ya tinggal naik kelas saja tanpa harus membayar biaya apapun itu bentuknya. Tapi koq di SMAN 1 BULULAWANG setiap kenaikan kelas harus bayar dl 700rb… seperti murid baru, bukannya anak saya sudah murid di sekolah tersebut kenapa harus daftar ulang ? ni kan sekolah negeri bukan sekolah swasta. Mungkin kalo swasta masi wajar ya kali ada pungutan liar. Karena bukan di Kelola pemerintah. Dan di setiap kali saya minta nota pembayaran daftar ulang selalu tidak di berikan atau bukti lampiran resmi dari sekolah jika memang ada biaya daftar ulang. 2. Di kenaikan kelas tahun ini 2020, spp di naikkan yang awalnya 125rb di tahun 2019 menjadi 200rb. Pihak sekolah beralasan kalua dana BOS dari pemerintah tidak cair maka meminta kesediaan dr wali murid membayar kenaikan spp sebesar 75rb (total spp 200rb) UNTUK MEMBIAYAI GURU HONORER dan akan di kembalikan jika dana BOS di adakan Kembali. Sedangkan saya confirm ke dinas Pendidikan bahwa dana BOS akan cair dalam beberapa hari ini. Kalau memang akan cair mengapa pihak SMAN 1 BULULAWANG tetap menaikkan spp ?! bukankah alasan menaikkan biaya spp itu karena dana BOS tidak ada. Dan lagi2 di setiap saya minta lampirn resmi dr pihak sekolah yang menyatakan kenaikan spp dikarenakan dana BOS tidak pernah di berikan. Mengapa pihak sekolah berbohong ? bukankah ini merugikan para wali murid dan menguntungkan pihak sekolah yang TIDAK BERTANGGUNG JAWAB ? saya selaku orang tua siswa dan mewakilkan suara wali murid lainnya merasa terbebani… karena kita terkesan mau tidak mau harus menerima keputusan dari sekolah yg sepihak itu. Bukan kah kita menyekolah anak kita di SMA Negeri supaya tidak begitu terbebani dengan biaya seperti SMA Swasta. Tapi mengapa SMA NEGERI 1 BULULAWANG itu seakan seperti SMA SWASTA yang selalu ada pungutan biaya TIDAK RESMI. Saya dan para wali murid memohon bantuan dengan sangat dalam hal ini, jika memang peraturan di atas tidak perna ada di dinas Pendidikan, mohon untuk segera di tindak lanjuti (seperti di audit atau Tindakan lainnya) supaya semua murid di SMAN 1 Bululawang tidak menjadi korban pungutan liar sekolah yang tidak bertanggung jawab. Agar SMAN 1 Bululawang bs menjadi selayaknya SMA NEGERI LAINNYA yang amanah jujur dan mengayomi para murid2nya. Tapi jika memang point2 di atas dikeluarkan resmi dari pemerintah. Kami mohon untuk bs memberikan surat lampiran resmi yg di keluarkan pihak SMAN 1 BULULAWANG dan berstempel agar kami memiliki bukti otentik sehingga kami tidak berpikir negative (seperti Pemerasan pihak sekolah) tentang sekolah ini. Terima Kasih banyak dan mohon dengan sangat bantuannya #SuarawalimuridSMAN1BULULAWANG'

In [10]:
predict_from_text(text)

The predicted category using the rocchio model is DINAS PENDIDIKAN KOTA MALANG.
The predicted category using the knn model is DINAS PENDIDIKAN KOTA MALANG.
